In [6]:
from hypergraphs import *
from kegg import *
import csv

In [77]:
deregulationDir="../../LaTeX/Data/KEGG-Pathways/AgeDeregulation/Glycol_First_Try/"
dataDirectory = "../../LaTeX/Data/KEGG-Pathways/"

organismDict = {"mmu": "Mus musculus (house mouse)", "dre": "Danio rerio (zebrafish)", "nfu": "Nothobranchius furzeri (turquoise killifish)"}
tissueList = ["brain", "liver", "skin"]

In [44]:
organism="nfu"
tissue="skin"

def getDeregulationMap(organism, tissues):
    """
        Get deregulation map for a given organism (e.g., "mmu") and a specific tissue (e.g., "skin") or list
        of tissues.
        The map has reactions as keys (e.g., R00103) and its values are a sequence of lists (all of the same size); 
        each list contains the deregulation values at specific ages.
    """
    der_map = {}
    if type(tissues) is not list:
        tissues = [tissues]
    for tissue in tissues:
        with open(os.path.join(deregulationDir,f"{organism}_RPKM_{tissue}_00010_Normalized.txt")) as f:
            reader = csv.reader(f, delimiter="\t")
            for line in reader:
                reaction = "rn" + line[0][3:9]   # Duplicate reactions are kept with duplicate names
                values = [float(t) for t in line[3:]]
                if reaction not in der_map:
                    der_map[reaction] = []
                der_map[reaction] += [values]
    return der_map

In [37]:
H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML([os.path.join(dataDirectory, organism, f"{organism}00010-sbml.xml")])
G, dd = hg2g(H)

In [39]:
centrality=nx.betweenness_centrality(G)


In [48]:
np.average([centrality[k] for k in sr2r if sr2r[k]=="rnR00658"])

0.08718354430379747

In [46]:
der_map = getDeregulationMap("nfu", "skin")

In [72]:
def average_deregulation(sr2r, centrality, der_map):
    """
        For a given sr2r map (map from subreaction names to reactions, e.g., 'rnR00710_R' -> 'rnR00710'),
        centrality (dictionary from reactions to centrality values), deregulation map (as returned by getDeregulationMap)
        produce the average centrality * deregulation for each time point.
    """
    n_age_points = len(der_map[next(iter(der_map))][0])
    t = np.array([0.0] * n_age_points)
    count = 0
    for k, listvalues in der_map.items():
        centr = np.average([centrality[kk] for kk in sr2r if sr2r[kk]==k]) # Centrality of reaction
        if np.isnan(centr):
            continue
        dereg = [np.average([v[i] for v in listvalues]) for i in range(n_age_points)] # Deregulation values (one for each age)
        s = centr*np.array(dereg)
        count += 1
        t += s
    return t/count

In [89]:
for organism, organism_name in organismDict.items():
    H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML([os.path.join(dataDirectory, organism, f"{organism}00010-sbml.xml")])
    G, dd = hg2g(H)
    centrality = nx.betweenness_centrality(G)
    avgd = average_deregulation(sr2r, centrality, getDeregulationMap(organism, "brain"))
    print(f"{organism_name:40.40}\t{avgd}")

Mus musculus (house mouse)              	[13.88287034 13.48423308 14.55791181 13.98416059 13.8065526 ]
Danio rerio (zebrafish)                 	[ 9.78108459  9.99116108 10.49646622  9.73770804  9.0851725 ]
Nothobranchius furzeri (turquoise killif	[8.55653031 8.37344154 8.33470044 7.58287193 7.97215917]
